<a href="https://colab.research.google.com/github/Abdallah-Alhasani10/chatbot-for-custom-website/blob/main/custom_chatbot_for_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [119]:
URLS=[
     'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]


In [120]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [121]:
loaders = UnstructuredURLLoader(urls=URLS)
data = loaders.load()

In [122]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [123]:
text_chunk=text_splitter.split_documents(data)

In [124]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectorstores=FAISS.from_documents(text_chunk,embedding)

In [125]:
from langchain.llms.base import LLM
from typing import Optional
from pydantic import BaseModel, Field, PrivateAttr
class LLMWapper(LLM, BaseModel):
    max_length: int = Field(100, description="Maximum length of generated text")
    _tokenizer: AutoTokenizer = PrivateAttr()
    _model: AutoModelForSeq2SeqLM = PrivateAttr()

    def __init__(self, tokenizer: AutoTokenizer, model: AutoModelForSeq2SeqLM, **kwargs):
        super().__init__(**kwargs)
        self._tokenizer = tokenizer
        self._model = model

    def _call(self, prompt: str, stop: Optional[list] = None, **kwargs) -> str:
        inputs = self._tokenizer(prompt, return_tensors="pt")
        outputs = self._model.generate(
            input_ids=inputs["input_ids"],
            max_length=self.max_length,
            num_return_sequences=1,
            pad_token_id=self._tokenizer.eos_token_id,
        )
        result = self._tokenizer.decode(outputs[0], skip_special_tokens=True)
        return result

    @property
    def _llm_type(self) -> str:
        return "custom"

In [126]:
llm=LLMWapper(tokenizer=tokenizer, model=model, max_length=100)

In [127]:
llm.predict("please Provide a concise summary of Book Harry potter")
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstores.as_retriever())

In [ ]:
while True:
  query=input(f"Prompt")
  if query =='exit':
    print('Exiting')
    sys.exit()
  if query:
    result=chain({"question": query},return_only_outputs=True)
    print(textwrap.fill(result['answer'],100))


Token indices sequence length is longer than the specified maximum sequence length for this model (1739 > 512). Running this sequence through the model will result in indexing errors


https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-
models-23e539522acb
90%* ChatGPT Quality.
